<a href="https://colab.research.google.com/github/MVFran/PyTorchForDeepLearning/blob/master/RedNeuronalConvolucional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
# En esta red neuronal, vamos a utilizar la base de datos MNIST
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [22]:
# Comvertimos las imagenes de MNIST es un tensor de 4 dimensiones (Número de imágenes, alto, ancho, color)
transform = transforms.ToTensor()

In [23]:
# Cargamos la base de datos
train_data = datasets.MNIST(root = "/CNN_data", train = True, download = True, transform = transform)

In [24]:
test_data = datasets.MNIST(root = "/CNN_data", train = False, download = True, transform = transform)

In [25]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /CNN_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [26]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /CNN_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [27]:
# Creamos un tamaño del batch para las imágenes (en este caso 10)
train_loader = DataLoader(train_data, batch_size = 10, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 10, shuffle = False)

In [28]:
# Definimos el modelo de CNN
conv1 = nn.Conv2d(1, 6, 3, 1)
conv2 = nn.Conv2d(6, 16, 3, 1)

In [29]:
# Tomamos una unica imágen del dataset
for i, (X_train, y_train) in enumerate(train_data):
  break

In [30]:
X_train.shape

torch.Size([1, 28, 28])

In [31]:
x = X_train.view(1, 1, 28, 28)

In [32]:
# Probamos la primera convolución
x = F.relu(conv1(x)) # Rectified Linear Unit como función de activación
x.shape              # El 1 indica que es una imagen, el 6 es el número de filtros y 26, 26 las nuevas dimensiones de la imagen

torch.Size([1, 6, 26, 26])

In [33]:
# Se hace max pooling
x = F.max_pool2d(x, 2, 2) # Kernel de 2 y stride de 2
x.shape

torch.Size([1, 6, 13, 13])

In [34]:
# Ahora, hacemos la segunda convolución
x = F.relu(conv2(x))
x.shape

torch.Size([1, 16, 11, 11])

In [35]:
# pooling
x = F.max_pool2d(x, 2, 2)
x.shape # El valor se redondea hacia abajo, ya que no podemos inventar datos para redondear hacia arriba

torch.Size([1, 16, 5, 5])